<H1>The Transportation Problem</H1>
* Consider a set of warehouses each with a given inventory of widgets, and a set of demand centers each with a given demand for widgets. 
* How many widgets should we ship from each warehouse to each demand center such that all demand is satisfied and shipping costs are minimized?

![Transportation Problem](transportation_problem.png)
<H3> Sets and Indices </H3>
* $i \in I$: Warehouses
* $j \in J$: Customers (demand centers)

<H3>Data</H3>
* $u_i$: capacity for warehouse $i$ (widgets)
* $d_j$: demand at demand center $j$ (widgets)
* $c_{ij}$: shipping cost from warehouse $i$ to customer $j$ ($/widget)

<H3>Decision Variables</H3>
* $x_{ij}$: number of widgets to ship from warehouse $i$ to customer $j$

<H3>Linear Programming Formulation</H3>
\begin{eqnarray}
\min_{x} && \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \;\; \mbox{(minimize shipping costs)} \nonumber \\
\mbox{s.t.} && \sum_{i \in I} x_{ij} = d_j,\;\;j \in J \;\; \mbox{(satisfy demand)}\nonumber \\
&& \sum_{j \in J} x_{ij} \le u_i,\;\;i \in I \;\; \mbox{(don't exceed capacity)} \nonumber \\
&& x_{ij} \ge 0, \;\;i \in I,\;j \in J \;\; \mbox{(ship nonnegative quantities)} \nonumber
\end{eqnarray}

<H3>Inputs</H3>

We'll consider a test data set with four demand centers that have the following demands:

In [1]:
demands = [15, 18, 14, 20]

And with five warehouses that have the following widget capacities:

In [2]:
capacities = [20, 22, 17, 19, 18]

In [3]:
warehouses = range(len(capacities))
customers = range(len(demands))
print (warehouses)

range(0, 5)


Finally, we'll need to know the per unit shipping costs between each warehouse and demand center.

In [4]:
ship_costs =  [[4000, 2500, 1200, 2200],
               [2000, 2600, 1800, 2600],
               [3000, 3400, 2600, 3100],
               [2500, 3000, 4100, 3700],
               [4500, 4000, 3000, 3200]]

<H3>Decision Variables</H3>

For each warehouse, customer pair, we'll need to decide how many units to ship. After adding those variables, we can think about how to generate the linear expressions needed to create the model.

In [5]:
import gurobipy as grb
model = grb.Model()
model.addVars(warehouses, customers, name='ship')

Using license file C:\Users\liaml\gurobi.lic
Academic license - for non-commercial use only


{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (3, 0): <gurobi.Var *Awaiting Model Update*>,
 (3, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 2): <gurobi.Var *Awaiting Model Update*>,
 (3, 3): <gurobi.Var *Awaiting Model Update*>,
 (4, 0): <gurobi.Var *Awaiting Model Update*>,
 (4, 1): <gurobi.Var *Awaiting Model Update*>,
 (4, 2): <gurobi.Var *Awaiting Model Update*>,
 (4, 3): <gurobi.Var *Awaiting Model Update*>}

<H3>Linear Expressions</H3>

This transportation model has two families of constraints. It is important to understand for each constraint which index set is being looped over, and which is being summed over. For demands, there exists one constraint per customer and each constraint involves a sum over the warehouses, and for supplies there exists one constraint per warehouse and each constraint involves a sum over the customers.

Let's create a test model and experiment with generating expressions out of the ship variables.

In [6]:
import gurobipy as grb
m = grb.Model()
warehouses = range(len(capacities))
customers = range(len(demands))
to_ship = m.addVars(warehouses, customers, name='ship')

m.update()
to_ship

{(0, 0): <gurobi.Var ship[0,0]>,
 (0, 1): <gurobi.Var ship[0,1]>,
 (0, 2): <gurobi.Var ship[0,2]>,
 (0, 3): <gurobi.Var ship[0,3]>,
 (1, 0): <gurobi.Var ship[1,0]>,
 (1, 1): <gurobi.Var ship[1,1]>,
 (1, 2): <gurobi.Var ship[1,2]>,
 (1, 3): <gurobi.Var ship[1,3]>,
 (2, 0): <gurobi.Var ship[2,0]>,
 (2, 1): <gurobi.Var ship[2,1]>,
 (2, 2): <gurobi.Var ship[2,2]>,
 (2, 3): <gurobi.Var ship[2,3]>,
 (3, 0): <gurobi.Var ship[3,0]>,
 (3, 1): <gurobi.Var ship[3,1]>,
 (3, 2): <gurobi.Var ship[3,2]>,
 (3, 3): <gurobi.Var ship[3,3]>,
 (4, 0): <gurobi.Var ship[4,0]>,
 (4, 1): <gurobi.Var ship[4,1]>,
 (4, 2): <gurobi.Var ship[4,2]>,
 (4, 3): <gurobi.Var ship[4,3]>}

We'll pick a customer first, say customer 0, and generate an expression for the number of units that customer received.

In [7]:
grb.quicksum(to_ship[warehouse, 0] for warehouse in warehouses)

<gurobi.LinExpr: ship[0,0] + ship[1,0] + ship[2,0] + ship[3,0] + ship[4,0]>

This fixes customer $c_0$, and sums over the warehouses $w_0,\ldots,w_4$. If this works for customer $c_0$, it should work for the rest of the customers if we add an outer loop.

In [8]:
for customer in customers:
    print (grb.quicksum(to_ship[warehouse, customer] for warehouse in warehouses))

<gurobi.LinExpr: ship[0,0] + ship[1,0] + ship[2,0] + ship[3,0] + ship[4,0]>
<gurobi.LinExpr: ship[0,1] + ship[1,1] + ship[2,1] + ship[3,1] + ship[4,1]>
<gurobi.LinExpr: ship[0,2] + ship[1,2] + ship[2,2] + ship[3,2] + ship[4,2]>
<gurobi.LinExpr: ship[0,3] + ship[1,3] + ship[2,3] + ship[3,3] + ship[4,3]>


We could also replicate this functionality with Gurobi's built in .sum() method, which works on tupledicts like the one that holds our variables.

In [9]:
for customer in customers:
    print(to_ship.sum('*', customer))

<gurobi.LinExpr: ship[0,0] + ship[1,0] + ship[2,0] + ship[3,0] + ship[4,0]>
<gurobi.LinExpr: ship[0,1] + ship[1,1] + ship[2,1] + ship[3,1] + ship[4,1]>
<gurobi.LinExpr: ship[0,2] + ship[1,2] + ship[2,2] + ship[3,2] + ship[4,2]>
<gurobi.LinExpr: ship[0,3] + ship[1,3] + ship[2,3] + ship[3,3] + ship[4,3]>


This should have generated a linear expression for each customer $c_0,\ldots,c_3$. If so, these are the LinExpr objects we need to construct demand constraints. The takeaway is that the set we are summing over should participate in the inner loop, and the set we are writing the constraint over should participate in the outer loop.

To generate expressions for the supply constraints, we'll need to fix a warehouse and sum over the customers, so we'll want customers on the inner loop and warehouses on the outer.

In [10]:
for warehouse in warehouses:
    print (grb.quicksum(to_ship[warehouse, customer] for customer in customers))

<gurobi.LinExpr: ship[0,0] + ship[0,1] + ship[0,2] + ship[0,3]>
<gurobi.LinExpr: ship[1,0] + ship[1,1] + ship[1,2] + ship[1,3]>
<gurobi.LinExpr: ship[2,0] + ship[2,1] + ship[2,2] + ship[2,3]>
<gurobi.LinExpr: ship[3,0] + ship[3,1] + ship[3,2] + ship[3,3]>
<gurobi.LinExpr: ship[4,0] + ship[4,1] + ship[4,2] + ship[4,3]>


Each LinExpr object we just generated should consider a single warehouse and sum over all customers.

<H3>Constraints</H3>

Generating the correct LinExpr objects for each demand and supply is most of the challenge. We can turn each LinExpr into a constraint by using the overloaded $<=$, $>=$, and $==$ operators.

For the demand constraints, we'll loop over the customers and sum over the warehouses.

Note: We are using Python's list comprehension syntax here, which puts the outer for loop inside the []. It is still the case that the outer loop iterates over the customer, and the inner loop sums over the warehouses.

In [11]:
def get_demand_constrs(model, to_ship, demands):
    return [model.addConstr(grb.quicksum(to_ship[warehouse, customer] for warehouse in warehouses) == demand,
                            name='demand.' + str(customer))
           for customer, demand in enumerate(demands)]

In [12]:
def get_demand_constrs(model, to_ship, demands):
    return model.addConstrs((to_ship.sum('*', customer) == demand for customer, demand in enumerate(demands)),
                            name='demand')

We can write a similar method to generate the capacity constraints.

In [13]:
def get_capacity_constrs(model, to_ship, capacities):
    return [model.addConstr(grb.quicksum(to_ship[warehouse, customer] for customer in customers) <= capacity,
                            name='capacity.' + str(warehouse))
            for warehouse, capacity in enumerate(capacities)]

In [14]:
def get_capacity_constrs(model, to_ship, capacities):
    return model.addConstrs((to_ship.sum(warehouse, '*') <= capacity for warehouse, capacity in enumerate(capacities)),
                            name='capacity')


<H3>Solving the Model</H3>

Now we have the building blocks needed to build and solve a transportation model. Let's put it all together.

In [15]:
import gurobipy as grb
GRB = grb.GRB
def solve_transportation_model(capacities, demands):
    model = grb.Model()
    warehouses = range(len(capacities))
    customers = range(len(demands))
    to_ship = model.addVars(warehouses, customers, name='ship')
    model.update()
    capacity_constrs = get_capacity_constrs(model, to_ship, capacities)
    demand_constrs = get_demand_constrs(model, to_ship, demands)
    model.setObjective(grb.quicksum(ship_costs[warehouse][customer]*to_ship[warehouse, customer]
                                for warehouse in warehouses
                                for customer in customers))
    model.optimize()
    if model.Status == GRB.OPTIMAL:
        for (warehouse, customer), var in sorted(to_ship.items()):
            if var.X > 1e-4:
                print ("Ship", var.X, "units from warehouse", warehouse, "to customer", customer)

In [16]:
solve_transportation_model(capacities, demands)

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 9 rows, 20 columns and 40 nonzeros
Model fingerprint: 0x0cf54468
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+03, 5e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.02s
Presolved: 9 rows, 20 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3580000e+05   3.200000e+01   0.000000e+00      0s
       4    1.5390000e+05   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.03 seconds
Optimal objective  1.539000000e+05
Ship 14.0 units from warehouse 0 to customer 2
Ship 6.0 units from warehouse 0 to customer 3
Ship 15.0 units from warehouse 1 to customer 0
Ship 7.0 units from warehouse 1 to customer 3
Ship 7.0 units from warehouse 2 to customer 3
Ship 18.0 units from warehouse 3 to customer 1


In [17]:
m = grb.Model()

In [18]:
x = m.addVar()

In [19]:
m.update()

In [20]:
m.addConstr(x <= 1)

<gurobi.Constr *Awaiting Model Update*>

In [21]:
m.addConstr(x >= 2)

<gurobi.Constr *Awaiting Model Update*>

In [22]:
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2 rows, 1 columns and 2 nonzeros
Model fingerprint: 0xed7596f8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Presolve time: 0.01s

Solved in 0 iterations and 0.02 seconds
Infeasible model


In [23]:
x.X

AttributeError: Unable to retrieve attribute 'X'